## Elementary Effects for the Rust Model

In [1]:
import os
import numpy as np
import pandas as pd
from functools import partial
# from multiprocessing import cpu_count
# Just as long as PR is not accepted.
from python.morris import elementary_effects
# Just as long as rust_model_morris is not defined in auxiliary.py.
from ruspy.model_code.demand_function import get_demand

# from python.auxiliary import rust_model_morris
from python.auxiliary import setup_rust_model_001

In [2]:
# Version 01.
def rust_model_morris(x, n_draws, trans_probs, init_dict_estimation, demand_dict):
    
    n_evaluations = n_draws
    
    # Adapt function to work with changed number of trans_probs as well.
    n_trans_probs = len(trans_probs)
    
    demand_inputs = np.zeros((n_evaluations, n_trans_probs +2))
    demand_inputs[:, :n_trans_probs] = trans_probs
    demand_inputs[:, n_trans_probs:] = x['value'][:]
    
    demand_output = np.zeros((n_evaluations, 1))
    
    get_demand_partial = partial(get_demand, init_dict=init_dict_estimation, demand_dict=demand_dict)
    def _get_demand_mapping(x):
        return get_demand_partial(demand_params=x).iloc[0]['demand']
    
    demand_output = np.array(list(map(_get_demand_mapping, demand_inputs)))
    
    return demand_output

In [3]:
# Version 02.
# Handle one sample at a time. Also: check that values in df are accessed in the right way.
def rust_model_morris(x, trans_probs, init_dict_estimation, demand_dict):
    
    # n_evaluations = n_draws
    
    # Adapt function to work with changed number of trans_probs as well.
    n_trans_probs = len(trans_probs)
    
    # demand_inputs now 1d array.
    demand_inputs = np.zeros(n_trans_probs +2)
    demand_inputs[:n_trans_probs] = trans_probs
    
    demand_inputs[n_trans_probs] = x['value'][0]
    demand_inputs[n_trans_probs + 1] = x['value'][1]
    
    # demand_output = np.zeros((n_evaluations, 1))
    
    # get_demand_partial = partial(get_demand, init_dict=init_dict_estimation, demand_dict=demand_dict)
    # def _get_demand_mapping(x):
    #     return get_demand_partial(demand_params=x).iloc[0]['demand']
    
    # demand_output = np.array(list(map(_get_demand_mapping, demand_inputs)))
    demand_output = get_demand(init_dict_estimation, demand_dict, demand_inputs).iloc[0]['demand']
    return demand_output

In [4]:
# Get model specs.
model_setup = setup_rust_model_001()
params = model_setup['params']
ev = model_setup['ev']
costs = model_setup['costs']
trans_probs = model_setup['trans_probs']
trans_mat = model_setup['trans_mat']
init_dict_simulation = model_setup['init_dict_simulation']
init_dict_estimation = model_setup['init_dict_estimation']
demand_dict = model_setup['demand_dict']

In [6]:
n_draws = 1000

# Choose largest num_sim for which I simulated cov and mean of RC/theta_11.
num_sim = 5000
cov_data = np.load(os.path.join('data', f'cov_rc_theta_11_{num_sim}.npy'))
# mean = np.load(os.path.join('data', f'mean_rc_theta_11_{num_sim}.npy'))

# Use mean (simulated) or params (true values)?
params = pd.DataFrame(data=params, columns=['value'])
cov = pd.DataFrame(data=cov_data)

model = partial(
    rust_model_morris, trans_probs=trans_probs, 
    init_dict_estimation=init_dict_estimation, demand_dict=demand_dict,
    )

In [7]:
%%time
# 1,000 samples take 20min 52s.
n_cores = os.cpu_count()
parallel = 'joblib'

morris_indices = elementary_effects(model, params, cov, n_draws, 'sobol', n_cores, parallel)

Wall time: 20min 52s


In [36]:
for key, value in morris_indices.items():
    print(key, '\n', '-----------', '\n', value, '\n')

mu_ind 
 ----------- 
 0    0.000000
1    8.626172
dtype: float64 

mu_corr 
 ----------- 
 0    193.595770
1    421.323963
dtype: float64 

sigma_ind 
 ----------- 
 0    0.000000
1    2.862762
dtype: float64 

sigma_corr 
 ----------- 
 0     69.841372
1    808.983145
dtype: float64 

mu_ind_cum 
 ----------- 
        0         1
1    NaN       NaN
2    0.0  6.425311
3    0.0  6.803662
4    0.0  6.954638
5    0.0  6.956427
..   ...       ...
96   0.0  8.652078
97   0.0  8.672290
98   0.0  8.662116
99   0.0  8.637241
100  0.0  8.626172

[100 rows x 2 columns] 

mu_corr_cum 
 ----------- 
               0           1
1           NaN         NaN
2    145.403936  134.101378
3    168.933786  115.343929
4    143.287394  115.365823
5    158.855970  120.408771
..          ...         ...
96   191.425946  425.724125
97   192.293764  422.091268
98   192.814843  422.629617
99   193.312184  423.760763
100  193.595770  421.323963

[100 rows x 2 columns] 

sigma_ind_cum 
 ----------- 
        0   